# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
import glob
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
from datetime import datetime, timedelta ,time 

# 写入excel的包
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Data_Clean, Risk_Data, Week_Model

In [2]:
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
clean = Data_Clean()
risk_data = Risk_Data()
week_models = Week_Model()

In [3]:
# 星期几映射
day_name_cn_mapping = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期日'
}

# 定义数据库连接

In [4]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [5]:
# df_order = pd.read_excel('E:/myfile/p站数据/当月数据/722-804.xlsx')
# df_order.shape
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status,om.order_method
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips   
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,pa.type
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tprm.purchase_amount, tprm.all_rental, tprm.order_number order_number2
,om.buy_service_product,tso.status as service_status 
, tprm.contract_price, tor.update_time, tomt.reason, om.order_type
from db_digua_business.t_order  om 
LEFT JOIN db_digua_business.t_postlease_receivables_monitoring tprm on om.id=tprm.order_id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')

-- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -16 day )              -- 近15天数据
-- and YEAR(om.create_time) in (2023, 2024)
-- and  HOUR(om.create_time)< 11
and date_format(om.create_time, '%Y-%m-%d') >= '2025-06-01'
-- and date_format(om.create_time, '%Y-%m-%d') <= '2025-02-28'
;
'''
df_order = query(sql1)
df_order.shape

(1118084, 43)

In [6]:
sql2 = '''
SELECT 
id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status  
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.rejected') end,'"','') as rejected
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.strategy_241011') end,'"','') as strategy_241011 
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.strategy_241205_rejected') end,'"','') as strategy_241205_rejected  
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.strategy_241212_rejected') end,'"','') as strategy_241212_rejected  
, mobile
FROM `db_credit`.`risk`
'''
df_risk = query(sql2)

In [7]:
sql_risk = ''' -- risk等级表
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r from db_credit.risk
'''
df_risks = query(sql_risk)
# df_risks.loc[:, 'time_r'] = pd.to_datetime(df_risks.time.dt.date,errors="coerce")
# df_risks = df_risks.sort_values(by='time', ascending=False).groupby('id_card_r').head(1)

In [8]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
# df_re = df_re.sort_values(by='time', ascending=False).groupby('id_card_re').head(1)

In [9]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

# 合并数据

In [10]:
df_order = df_order[df_order.type != 4]
df = df_order.copy()
df.shape

(1118084, 43)

## 日期处理

In [11]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df['下单月份'] = df['create_time'].dt.strftime('%Y-%m')

## 备注信息处理 及关键分类 自段处理

In [12]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")


df.loc[:,"颜色"]=df.apply(lambda x: clean.getcolor(x["sku_attributes"]),axis=1)
df.loc[:,"内存"]=df.apply(lambda x: clean.getneicun(x["sku_attributes"]),axis=1)
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

## 判断进件

In [13]:
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")

## 判断渠道归属

In [14]:

df.loc[:,"归属渠道"]=df.apply(lambda x:clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
conditions_1 = ['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']
# 租机宝 、 富士康租机 、 兔丫租赁 、 邦客惠乘车码  -- 不算S量
df['归属渠道'] = np.where(df.归属渠道.isin(conditions_1), 'S量', df.归属渠道)
df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)

## 订单预排序去重  剔除内部订单

In [15]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

## 定义审核状态

In [16]:
# note 2025-02-24添加
df = df.merge(df_risks[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')

# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)
df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)
# 定义各个节点的状态
df = clean.status_node(df)

# 去除商家数据

In [17]:
#剔除商家数据只保留自营租机业务数据
df = clean.drop_merchant(df)
# 剔除拒量数据
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]

# df：总体
# df2：不含拒量、不含小蚂蚁，即机审通过的订单

# 导出数据 去重订单明细  

# 1.总体转化数据情况

In [18]:
#总体
df_all = all_models.data_group(df, df2, df_risk_examine, '下单日期')
df_all = df_all[["去重订单数", "前置拦截", "拦截率", "进件数", "预授权通过率", "机审强拒", "强拒比例", "机审通过件", "风控通过件","风控通过率", "出库前风控强拒", "出库前强拒比例", "出库", "进件出库率", "订单出库率"]].reset_index()
# df_all
df_all.insert(0, '星期', df_all['下单日期'].apply(lambda x: day_name_cn_mapping[x.day_name()]))
df_all.insert(1, '月份', df_all['下单日期'].astype(str).str.split('-').str[0]+'-'+df_all['下单日期'].astype(str).str.split('-').str[1])

## 11.芝麻租物渠道转化

In [19]:
df_zw1 = df[df["归属渠道"]=="芝麻租物"]
df_zw2 = df2[df2["归属渠道"]=="芝麻租物"]
df_zw_g = all_models.data_group(df_zw1, df_zw2, df_risk_examine, '下单日期').reset_index()
df_zw_g = df_zw_g[['下单日期', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","出库","进件出库率","订单出库率"]]
# df_zw_g

## 12.搜索渠道转化情况

In [20]:
df_ss1 = df[df["归属渠道"]=="搜索渠道"]
df_ss2 = df2[df2["归属渠道"]=="搜索渠道"]
df_ss_g = all_models.data_group(df_ss1, df_ss2, df_risk_examine, '下单日期').reset_index()
df_ss_g = df_ss_g[['下单日期', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","出库","进件出库率","订单出库率"]]


## 13.单人聊天会话中的小程序消息卡片（分享

In [21]:
df_dr1 = df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df_dr2 = df2[df2["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df_dr_g = all_models.data_group(df_dr1, df_dr2, df_risk_examine, '下单日期')
df_dr_g = clean.continuous_dates(df_dr_g).reset_index(names=['下单日期'])
df_dr_g = df_dr_g[['下单日期', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","出库","进件出库率","订单出库率"]]


## 14.S量转化

In [22]:
df_sl1 = df[df["归属渠道"]=="S量"]
df_sl2 = df2[df2["归属渠道"]=="S量"]
df_sl_g = all_models.data_group(df_sl1, df_sl2, df_risk_examine, '下单日期')
df_sl_g = clean.continuous_dates(df_sl_g).reset_index(names=['下单日期'])
df_sl_g = df_sl_g[['下单日期', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","出库","进件出库率","订单出库率"]]


In [23]:
df_sl_g[(df_sl_g.下单日期=='2025-01-27')][['去重订单数']]

,去重订单数


## 15.抖音

In [24]:
df_dy1 = df[df["归属渠道"]=="抖音渠道"]
df_dy2 = df2[df2["归属渠道"]=="抖音渠道"]
df_dy = all_models.data_group(df_dy1, df_dy2, df_risk_examine, '下单日期')
df_dy_g = clean.continuous_dates(df_dy).reset_index(names=['下单日期'])
df_dy_g = df_dy_g[['下单日期', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","出库","进件出库率","订单出库率"]]
# df_dy_g

## 16.二手非抖音

In [25]:
df_esfdy1 = df[(df["归属渠道"]!="抖音渠道")&(df.是否二手 == 1)]
df_esfdy2 = df2[(df2["归属渠道"]!="抖音渠道")&(df2.是否二手 == 1)]
df_esfdy_g = all_models.data_group(df_esfdy1, df_esfdy2, df_risk_examine, '下单日期').reset_index()
df_esfdy_g = df_esfdy_g[['下单日期', "去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","出库","进件出库率","订单出库率"]]


## 17.拒量

In [26]:
df_jl = df[df["是否拒量"]== 1]
# 拒量池	机审强拒	机审强拒比例	出库前风控强拒	出库前强拒比例	出库	进件出库率
df_jl_group = df_jl.groupby('下单日期').agg({'是否进件': 'sum', '机审强拒': 'sum', '出库前风控强拒': 'sum', '是否出库': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库': '出库'}).reset_index()
df_jl_group.loc[:, '机审强拒比例'] = (df_jl_group.机审强拒/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '出库前风控强拒比例'] = (df_jl_group.出库前风控强拒/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '进件出库率'] = (df_jl_group.出库/df_jl_group.进件数).map(lambda x: format(x, '.2%'))

df_jl_group = df_jl_group[['下单日期', '进件数', '机审强拒', '机审强拒比例', '出库前风控强拒', '出库前风控强拒比例', '出库', '进件出库率']]
# df_jl_group

## 18.京享租

In [27]:
df_jd1 = df[(df["归属渠道"]=="京东渠道")]
df_jd2 = df2[(df2["归属渠道"]=="京东渠道")]
df_jd1 = df_jd1[df_jd1.下单日期>='2025-05-20']
df_jd2 = df_jd2[df_jd2.下单日期>='2025-05-20']
df_jd_g = all_models.data_group(df_jd1, df_jd2, df_risk_examine, '下单日期')
df_jd_g = df_jd_g[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","出库","进件出库率","订单出库率"]]
df_jd_g = clean.continuous_dates(df_jd_g).rename_axis('下单日期').reset_index()
# df_jd_g

In [28]:
qd_all = pd.concat([df_all, df_zw_g.iloc[:, 1:], df_ss_g.iloc[:, 1:], df_dr_g.iloc[:, 1:], df_sl_g.iloc[:, 1:], df_dy_g.iloc[:, 1:], df_esfdy_g.iloc[:, 1:], df_jl_group.iloc[:, 1:]], axis=1)
qd_all = qd_all.merge(df_jd_g, on='下单日期', how='left')

# 2.免审免押转化

## 21.免审订单转化统计

In [29]:
# 免审
df_ms1 = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2 = df2[(df.is_vip=='1')&(df.status_result=='0')]
df_ms_g = all_models.data_group(df_ms1, df_ms2, df_risk_examine, '下单日期')
df_ms2_g = df_ms_g[["去重订单数", "前置拦截", "进件数", "机审强拒", "机审通过件", '人审拒绝', "风控通过件", '客户取消', '无法联系', "出库前风控强拒", '待审核', "出库"]]
df_ms_g = df_ms_g[["进件数","人审拒绝","客户取消","出库","待审核","进件出库率","取消率","人审拒绝率",'出库前风控强拒','出库前强拒比例','无法联系','无法联系占比']].reset_index()

df_ms_g.insert(0, '星期', df_ms_g['下单日期'].apply(lambda x: day_name_cn_mapping[x.day_name()]))
df_ms_g.insert(1, '月份', df_ms_g['下单日期'].astype(str).str.split('-').str[0]+'-'+df_ms_g['下单日期'].astype(str).str.split('-').str[1])

## 22.全免押

In [30]:
#全免押
df_my1 = df[df["押金类型"]=="全免押"] 
df_my2 = df2[df2["押金类型"]=="全免押"] 
df_my_g = all_models.data_group(df_my1, df_my2, df_risk_examine, '下单日期')
df_my_g = df_my_g[["去重订单数","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝",
                    "风控通过件","风控通过率","客户取消","待审核","出库","进件出库率","取消率","人审拒绝率","订单出库率"]].reset_index()


## 23.部分免押

In [31]:
#部分免押
df_fmy1 = df[df["押金类型"]=="部分免押"]
df_fmy2 = df2[df2["押金类型"]=="部分免押"]
df_fmy_g = all_models.data_group(df_fmy1, df_fmy2, df_risk_examine, '下单日期')
df_fmy2_g = df_fmy_g[["去重订单数", "进件数", "机审强拒", "机审通过件", '人审拒绝', "风控通过件", '客户取消', '待审核', "出库"]]
df_fmy_g = df_fmy_g[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","取消率","人审拒绝率"]].reset_index()


In [32]:
# 免审 免押 非免押
m_all = pd.concat([df_ms_g, df_my_g, df_fmy_g], axis=1)

# 3.风控拦截

In [33]:
def qzlj(df, df_old, list_reg, key=None):
    '''
    统计前置拦截的每天数据和周数据
    df：已经统计好的前置拦截数据
    df_old：已经统计好的每天数据或每周的数据
    list_reg：前置拦截的列名
    key：判断是插入每天的列数据还是每周的列数据
    '''
    df.insert(1 , '去重订单数', df_old.去重订单数)
    df.insert(2 , '拦截订单数', df_old.前置拦截)
    df.loc[:, '拦截率'] = (df.拦截订单数/df.去重订单数).map(lambda x: format(x, '.2%'))
    # 判断是周数据还是每天数据获取对应的列名数据比例
    if key=='week':
        df_new = df[list_reg].divide(df.去重订单数, axis=0).fillna(0).map(lambda x: format(x, '.2%'))
    else:
        df_new = df[list_reg[1:]].divide(df.去重订单数, axis=0).fillna(0).map(lambda x: format(x, '.2%'))
    # 对获取的比例的列名进行重命名
    for col in df_new.columns:
        df_new.rename(columns={f'{col}': f'{col}_new'}, inplace=True)
    # 根据key选择插入的是周数据还是天数据
    if key is None:
        df_new.insert(0, '下单日期', df_old.下单日期)
        df_new.insert(0, '星期', df_new['下单日期'].apply(lambda x: day_name_cn_mapping[x.day_name()]))
        df_new.insert(1, '月份', df_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_new['下单日期'].astype(str).str.split('-').str[1])
    else:
        df_new.insert(0, 'week_group', df.week_group)
        df_new.insert(1, 'nan1', np.nan)
        df_new.insert(2, 'nan2', np.nan)
    df_new.insert(3, '拦截率', df.拦截率)
    # 对原始数据和比例数据进行拼接
    df_news = pd.concat([df_new, df.iloc[:, 1:-1]], axis=1)
    return df_news

# 命中非免押用户拒绝	命中法院失信/限高被执行人	命中特殊名单验证-高风险强拒	命中模型强拒	命中蚁盾分强拒	命中融安分强拒	评分等级低于D	命中云商分强拒	命中青云分强拒	命中策略策略240703_4强拒	命中策略240703_14强拒	命中策略240703_10强拒	综合风险等级为3	非芝麻租物履约历史等级=1强拒	命中百融借贷风险勘测强拒	评分等级D且综合风险等级为3	30天多头>13且90天多头>39	评分等级D且非免押客户	命中策略strategy_240801强拒	命中刑事案件	命中借贷纠纷	命中蚂蚁数控风险等级强拒	命中策略strategy_240927强拒	命中FICO分强拒	命中银联模型、FICO、青云分组合规则强拒	命中抖音_240829策略强拒
def all_js(df, df_old, list_js, key=None, model='下单日期'):
    '''
    统计机审强拒和出出库前强拒的每天和周数据
    df：已经统计好的机审强拒和出库前强拒的数据
    df_old：已经统计好的每天的数据或每周的数据
    list_js：机审强拒和出库前强拒的列名
    key：判断是插入每天的列数据还是每周的列数据
    model：选择是通过哪个字段进行关联
    '''
    # 对统计好的天数据或周数据和统计好的机审强拒和出库前强拒的数据进行关联
    df2 = df.merge(df_old[[model,'进件数', '机审强拒', '出库前风控强拒']], on=model, how='left').rename(columns={'进件数':'进件', '机审强拒': '机审强拒订单数', '出库前风控强拒':'出库前强拒订单数'})
    # 获取对应列名的数据比例
    df_new = df2[list_js].divide(df2.进件, axis=0).fillna(0).map(lambda x: np.nan if x==0 else format(x, '.2%'))
    # 对获取的比例的列名进行重命名
    for col in df_new.columns:
        df_new.rename(columns={f'{col}': f'{col}_new'}, inplace=True)
    # 根据key判断插入的是周数据还是天数据
    if key is None:
        df_new.insert(0, '下单日期', df2.下单日期)
        df_new.insert(0, '星期', df_new['下单日期'].apply(lambda x: day_name_cn_mapping[x.day_name()]))
        df_new.insert(1, '月份', df_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_new['下单日期'].astype(str).str.split('-').str[1])
    else:
        df_new.insert(0, 'week_group', df2.week_group)
        df_new.insert(1, 'nan1', np.nan)
        df_new.insert(2, 'nan2', np.nan)
    df_new.insert(3, '进件', df2.进件)
    df_new.insert(4, '机审强拒订单数', df2.机审强拒订单数)
    df_new.insert(5, '出库前强拒订单数', df2.出库前强拒订单数)
    df_new.insert(6, '机审强拒比例', (df_new.机审强拒订单数/df_new.进件).map(lambda x: format(x, '.2%')))
    df_new.insert(7, '出库前强拒比例', (df_new.出库前强拒订单数/df_new.进件).map(lambda x: format(x, '.2%')))
    df_new.insert(8, '出库前强拒占比', (df_new.出库前强拒订单数/(df_new.进件-df_new.机审强拒订单数)).map(lambda x: format(x, '.2%')))
    df_new.insert(9, '进件后风控强拒比例', ((df_new.机审强拒订单数+df_new.出库前强拒订单数)/df_new.进件).map(lambda x: format(x, '.2%')))
    # 对列名比例和原始数据进行拼接
    df_news = pd.concat([df_new, df2[list_js]], axis=1)
    return df_news


def week_reg(df, df_week, list_reg, key):
    '''
    df：已经统计好的前置拦截或机审强拒或出库前强拒的数据
    df_week：已经统计好的周数据
    list_reg：前置拦截或机审强拒或出库前强拒的列名
    key：判断是前置拦截还是机审强拒
    '''
    df_reg_week = df.set_index('下单日期')[list_reg].fillna(0)
    # 对数据进行汇总
    week = week_models.W_group(week_models.custom_weekly_resampler(df_reg_week, 7)).iloc[:, :-1]
    if key=='前置':
        week_new = qzlj(week, df_week, list_reg, key='week')
    elif key=='机审':
        week_new = all_js(week, df_week, list_reg, key='week', model='week_group')
    return week_new

def update_col(df, key=None):
    df = df.fillna(0)
    if key=='前置':
        df.loc[:, '司法强拒_前置'] = np.where(("命中法院失信/限高被执行人" in df.columns)|("命中借贷纠纷" in df.columns)|("命中刑事案件" in df.columns)|("命中高风险名单" in df.columns), df['命中法院失信/限高被执行人']+df.命中借贷纠纷+df.命中刑事案件+df.命中高风险名单, 0)
    else:
        df.loc[:, '司法强拒_机审'] = np.where(("命中法院失信/限高被执行人" in df.columns)|("命中特殊名单验证-高风险强拒" in df.columns)|('命中刑事案件_机审' in df.columns)|('命中借贷纠纷_机审' in df.columns), df['命中法院失信/限高被执行人']+df['命中特殊名单验证-高风险强拒']+df.命中刑事案件_机审+df.命中借贷纠纷_机审, 0)
        df.loc[:, '司法强拒_出库前'] = np.where(("命中刑事案件_出库前" in df.columns)|("命中借贷纠纷_出库前" in df.columns), df.命中借贷纠纷_出库前+df.命中刑事案件_出库前, 0)
    return df

## 31.前置拦截

### 31_0.前置拦截-all

In [65]:
df_pre_reject_all = risk_data.pre_reject(df, '下单日期')
df_pre_reject_all = update_col(df_pre_reject_all, '前置')
# 冰鉴火眸分<500(7.1之前为智融分低于458)	命中法院失信/限高被执行人	年龄超过49岁或低于18岁	命中强拒加入临时黑名单	命中特殊地区 新疆|西藏	命中高风险名单	当月下单次数大于等于10次	命中借贷纠纷	命中刑事案件	当日下单次数>=5次	命中融安分低于680	身份证号码命中黑名单
list_reg=['下单日期', "冰鉴火眸分<500","司法强拒_前置","年龄超过49岁或低于18岁","命中强拒加入临时黑名单","命中特殊地区 新疆|西藏","当月下单次数大于等于10次", "当日下单次数大于等于5次","命中融安分低于680","身份证命中黑名单",'评分等级低于D','命中TD212强拒']

df_pre_reject_all = df_pre_reject_all[list_reg]
qz_all = qzlj(df_pre_reject_all, df_all, list_reg)
# qz_all

,星期,月份,下单日期,拦截率,冰鉴火眸分<500_new,司法强拒_前置_new,年龄超过49岁或低于18岁_new,命中强拒加入临时黑名单_new,命中特殊地区 新疆|西藏_new,当月下单次数大于等于10次_new,当日下单次数大于等于5次_new,命中融安分低于680_new,身份证命中黑名单_new,评分等级低于D_new,命中TD212强拒_new,去重订单数,拦截订单数,冰鉴火眸分<500,司法强拒_前置,年龄超过49岁或低于18岁,命中强拒加入临时黑名单,命中特殊地区 新疆|西藏,当月下单次数大于等于10次,当日下单次数大于等于5次,命中融安分低于680,身份证命中黑名单,评分等级低于D,命中TD212强拒
0,星期日,2025-06,2025-06-01,30.87%,7.11%,6.15%,1.65%,2.30%,1.20%,0.00%,0.00%,0.00%,0.03%,4.25%,7.73%,3997,1234,284,246,66,92,48,0.0,0.0,0.0,1.0,170,309
1,星期一,2025-06,2025-06-02,30.54%,7.62%,6.09%,1.76%,2.73%,1.62%,0.00%,0.00%,0.00%,0.00%,3.60%,7.39%,5170,1579,394,315,91,141,84,0.0,0.0,0.0,0.0,186,382
2,星期二,2025-06,2025-06-03,27.43%,7.39%,5.38%,1.32%,2.37%,1.67%,0.00%,0.00%,0.00%,0.02%,3.72%,6.03%,5916,1623,437,318,78,140,99,0.0,0.0,0.0,1.0,220,357
3,星期三,2025-06,2025-06-04,27.75%,7.16%,6.01%,1.46%,2.43%,1.41%,0.00%,0.00%,0.00%,0.00%,4.07%,5.67%,5895,1636,422,354,86,143,83,0.0,0.0,0.0,0.0,240,334
4,星期四,2025-06,2025-06-05,27.82%,7.52%,5.77%,1.33%,2.37%,1.59%,0.00%,0.00%,0.00%,0.00%,3.92%,5.88%,5769,1605,434,333,77,137,92,0.0,0.0,0.0,0.0,226,339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,星期四,2025-07,2025-07-31,33.73%,7.84%,8.41%,1.63%,3.62%,2.04%,0.00%,0.00%,0.00%,0.00%,3.45%,5.90%,6952,2345,545,585,113,252,142,0.0,0.0,0.0,0.0,240,410
61,星期五,2025-08,2025-08-01,33.40%,7.91%,7.23%,1.21%,4.59%,1.89%,0.00%,0.00%,0.00%,0.00%,2.96%,5.99%,7341,2452,581,531,89,337,139,0.0,0.0,0.0,0.0,217,440
62,星期六,2025-08,2025-08-02,33.25%,6.86%,7.84%,1.24%,4.40%,2.15%,0.00%,0.00%,0.00%,0.00%,2.79%,6.02%,6228,2071,427,488,77,274,134,0.0,0.0,0.0,0.0,174,375
63,星期日,2025-08,2025-08-03,34.42%,7.63%,8.06%,1.01%,4.47%,1.90%,0.00%,0.00%,0.00%,0.02%,2.95%,5.87%,6067,2088,463,489,61,271,115,0.0,0.0,0.0,1.0,179,356


### 31_1.芝麻渠道

In [35]:
df_zm = df[df.归属渠道=='芝麻租物']
df_pre_reject_zm = risk_data.pre_reject(df_zm, '下单日期')
df_pre_reject_zm = update_col(df_pre_reject_zm, '前置')
df_pre_reject_zm = df_pre_reject_zm[list_reg]
qz_zm = qzlj(df_pre_reject_zm, df_zw_g, list_reg)
# qz_zm

### 31_2.搜索渠道

In [36]:
df_ss = df[df.归属渠道=='搜索渠道']
df_pre_reject_ss = risk_data.pre_reject(df_ss, '下单日期')
df_pre_reject_ss = update_col(df_pre_reject_ss, '前置')
df_pre_reject_ss = df_pre_reject_ss[list_reg[:-2]]
qz_ss = qzlj(df_pre_reject_ss, df_ss_g, list_reg[:-2])
# qz_ss

### 31_3.单人会话

In [37]:
df_dr = df[df.归属渠道=='单人聊天会话中的小程序消息卡片（分享）']
df_pre_reject_dr = risk_data.pre_reject(df_dr, '下单日期')
df_pre_reject_dr = update_col(df_pre_reject_dr, '前置')
df_pre_reject_dr = df_pre_reject_dr[list_reg[:-2]]
qz_dr = qzlj(df_pre_reject_dr, df_dr_g, list_reg[:-2])
# qz_dr

### 31_4.S量

In [38]:
df_sl = df[df.归属渠道=='S量']
df_pre_reject_sl = risk_data.pre_reject(df_sl, '下单日期')
df_pre_reject_sl = clean.continuous_dates(df_pre_reject_sl.set_index('下单日期')).reset_index(names=['下单日期'])
df_pre_reject_sl = update_col(df_pre_reject_sl, '前置')
df_pre_reject_sl = df_pre_reject_sl[list_reg[:-2]]
qz_sl = qzlj(df_pre_reject_sl, df_sl_g, list_reg[:-2])
# qz_sl

### 31_5.抖音

In [39]:
df_dy = df[df.归属渠道=='抖音渠道']
df_pre_reject_dy = risk_data.pre_reject_dy(df_dy, '下单日期').set_index('下单日期')
df_pre_reject_dy = update_col(df_pre_reject_dy, '前置')
# 对不连续的日期进行填充
df_pre_reject_dy = clean.continuous_dates(df_pre_reject_dy).reset_index(names=['下单日期'])
df_pre_reject_dy = df_pre_reject_dy[list_reg[:-2]]
qz_dy = qzlj(df_pre_reject_dy, df_dy_g, list_reg[:-2])
# qz_dy

### 31_6.二手非抖音

In [40]:
df_esfdy = df[(df["归属渠道"]!="抖音渠道")&(df.是否二手 == 1)]
df_pre_reject_esfdy = risk_data.pre_reject(df_esfdy, '下单日期')
df_pre_reject_esfdy = update_col(df_pre_reject_esfdy, '前置')
df_pre_reject_esfdy = df_pre_reject_esfdy[list_reg[:-2]]
qz_esfdy = qzlj(df_pre_reject_esfdy, df_esfdy_g, list_reg[:-2])
# qz_esfdy

### 31_7.京享租

In [41]:
df_jd = df[(df["归属渠道"]=="京东渠道")]
df_jd = df_jd[df_jd.下单日期>='2025-05-20']
df_pre_reject_jd = risk_data.pre_reject(df_jd, '下单日期')
df_pre_reject_jd = update_col(df_pre_reject_jd, '前置')
df_pre_reject_jd = df_pre_reject_jd[list_reg[:-2]]
qz_jd = qzlj(df_pre_reject_jd, df_jd_g, list_reg[:-2])
# qz_jd

## 32.机审出库前强拒

### 32_0.机审出库前强拒-all

In [42]:
# 匹配小蚂蚁数据
df_xmy = df[(df['merchant_name'].isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机']))]
# 机审强拒数据重命名
df_risk.rename(columns={'time': 'time_risk', 'status': 'status_risk', 'rejected': 'rejected_risk', 'strategy_241205_rejected': 'strategy_241205_rejected_risk', 'strategy_241212_rejected': 'strategy_241212_rejected_risk', 'strategy_241011': 'strategy_241011_risk'}, inplace=True)
# 将拒绝理由统计在一起
df_risk['rejected_risk'] = np.where((df_risk.rejected_risk == '[]') | (df_risk.rejected_risk.isna()), 
                                    np.where((df_risk.strategy_241011_risk == '[]') | (df_risk.strategy_241011_risk.isna()),
                                        np.where((df_risk.strategy_241205_rejected_risk == '[]') | (df_risk.strategy_241205_rejected_risk.isna()),df_risk.strategy_241212_rejected_risk, df_risk.strategy_241205_rejected_risk),
                                            df_risk.strategy_241011_risk), 
                                    df_risk.rejected_risk)
df_risk['拒绝理由_risk'] = df_risk["rejected_risk"].str.replace("[","").str.replace("]","").str.replace('"','')
# 对小蚂蚁数据和机审强拒数据进行拼接
df_risk_all = pd.merge(df_xmy, df_risk, left_on='id_card_num', right_on='id_card', how='inner')
# 计算出库前强拒的订单数
df_risk_all2 = df_risk_all[(df_risk_all['time_risk'] <= df_risk_all['update_time'])]
# 进行排序并取到最近的一个订单
df_risk_all2 = df_risk_all2.sort_values('time_risk', ascending=False).groupby('order_id').head(1)
df_risk_all2 = df_risk_all2[df_risk_all2['status_risk'] == '1']

In [43]:
df_j = df[(df["进件"]=="进件")]
df_strategy = df_risk[((df_risk.strategy_241011_risk.notnull())&(df_risk.strategy_241011_risk!='[]'))|(df_risk.strategy_241205_rejected_risk.notna())]
df_j_new = df_j.merge(df_strategy, left_on='id_card_num', right_on='id_card', how='left')
df_j_new.drop_duplicates(subset=['order_number'], inplace=True)
df_j_new['拒绝理由'] = np.where((df_j_new.拒绝理由=='')&(df_j_new.拒绝理由_risk.notna()), df_j_new.拒绝理由_risk, df_j_new.拒绝理由)

In [44]:
# 对risk取出的拒绝理由进行替换
df_j_risk = df_j_new.iloc[:, :-1].merge(df_risk_all2[['order_number', '拒绝理由_risk']], on='order_number', how='left')
df_j_risk['拒绝理由'] = np.where(((df_j_risk['拒绝理由'].isna())|(df_j_risk.拒绝理由=='')) & df_j_risk['拒绝理由_risk'].notna(), df_j_risk['拒绝理由_risk'], df_j_risk['拒绝理由'])
# df_j_risk[(df_j_risk['机审强拒']==1)|(df_j_risk['出库前风控强拒']==1)][['create_time', 'order_number', '拒绝理由']].to_excel('F:/需求/二手手机/强拒原因.xlsx')

list_js = ['命中非免押用户拒绝', '司法强拒_机审','命中模型强拒','命中蚁盾分强拒','命中融安分强拒','评分等级低于D','命中云商分强拒','命中青云分强拒','命中策略策略240703_4强拒','命中策略240703_14强拒','命中策略240703_10强拒','综合风险等级为3','非芝麻租物进件且履约历史等级=1强拒','命中百融借贷风险勘测强拒','评分等级D且综合风险等级为3','30天多头>13且90天多头>39','评分等级D且非免押客户','命中策略strategy_240801强拒','司法强拒_出库前','命中蚂蚁数控风险等级强拒','命中策略strategy_240927强拒', '命中极信sc32007分强拒', '命中银联模型及Fico联合规则强拒', '命中抖音_240829策略强拒']
list_js2 = ['命中非免押用户拒绝','司法强拒_机审','命中模型强拒','命中蚁盾分强拒','命中融安分强拒','评分等级低于D','命中云商分强拒','命中青云分强拒','命中策略策略240703_4强拒','命中策略240703_14强拒','命中策略240703_10强拒','综合风险等级为3','非芝麻租物进件且履约历史等级=1强拒','命中百融借贷风险勘测强拒','评分等级D且综合风险等级为3','30天多头>13且90天多头>39','评分等级D且非免押客户','命中策略strategy_240801强拒','司法强拒_出库前','命中蚂蚁数控风险等级强拒','命中策略strategy_240927强拒', '命中极信sc32007分强拒', '命中银联模型及Fico联合规则强拒', '命中抖音_240829策略强拒']

df_model_reject_all_js = risk_data.model_reject2(df_j_risk, '下单日期')
df_model_reject_all_js = update_col(df_model_reject_all_js)
df_model_reject_all_js.sort_values(by=["下单日期"],inplace=True)
js_all = all_js(df_model_reject_all_js, df_all, list_js2)
# js_all

### 32_1.芝麻租物

In [45]:

df_zm=df_j_risk[(df_j_risk["归属渠道"]=="芝麻租物")&(df_j_risk["进件"]=="进件")]
df_model_reject_zm = risk_data.model_reject2(df_zm, '下单日期')
df_model_reject_zm = update_col(df_model_reject_zm)
df_model_reject_zm.sort_values(by=["下单日期"],inplace=True)
js_zm = all_js(df_model_reject_zm, df_zw_g, list_js2)
# js_zm

### 32_2.搜索渠道

In [46]:

df_ss=df_j_risk[(df_j_risk["归属渠道"]=="搜索渠道")&(df_j_risk["进件"]=="进件")]
df_model_reject_ss = risk_data.model_reject2(df_ss, '下单日期')
df_model_reject_ss = update_col(df_model_reject_ss)
df_model_reject_ss.sort_values(by=["下单日期"],inplace=True)
js_ss = all_js(df_model_reject_ss, df_ss_g, list_js)
# js_ss

### 32_3.单人会话

In [47]:

df_dr=df_j_risk[(df_j_risk["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）")&(df_j_risk["进件"]=="进件")]
df_model_reject_dr = risk_data.model_reject2(df_dr, '下单日期')
df_model_reject_dr = update_col(df_model_reject_dr)
df_model_reject_dr.sort_values(by=["下单日期"],inplace=True)
js_dr = all_js(df_model_reject_dr, df_dr_g, list_js)
# js_dr

### 32_4.S量

In [48]:

list_s = ['命中非免押用户拒绝','司法强拒_机审','命中模型强拒','命中蚁盾分强拒','命中融安分强拒','评分等级低于D','命中云商分强拒','命中青云分强拒','命中策略策略240703_4强拒','命中策略240703_14强拒','命中策略240703_10强拒','综合风险等级为3','非芝麻租物进件且履约历史等级=1强拒','命中百融借贷风险勘测强拒','评分等级D且综合风险等级为3','30天多头>13且90天多头>39','评分等级D且非免押客户','命中策略strategy_240801强拒','命中蚁盾分2.0强拒','命中占融202098联合规则强拒','命中占融202100联合规则强拒','命中策略strategy_b004强拒','司法强拒_出库前','命中蚂蚁数控风险等级强拒','命中策略strategy_240927强拒', '命中极信sc32007分强拒', '命中银联模型及Fico联合规则强拒', '命中抖音_240829策略强拒']
df_sl=df_j_risk[(df_j_risk["归属渠道"]=="S量")&(df_j_risk["进件"]=="进件")]
df_model_reject_sl = risk_data.model_reject2(df_sl, '下单日期')
df_model_reject_sl = update_col(df_model_reject_sl)
df_model_reject_sl.sort_values(by=["下单日期"],inplace=True)
js_sl = all_js(df_model_reject_sl, df_sl_g, list_s)
# js_sl

### 32_5.二手非抖音

In [49]:

df_esfdy=df_j_risk[(df_j_risk["归属渠道"] != "抖音渠道")&(df_j_risk.是否二手 == 1)&(df_j_risk["进件"]=="进件")]
df_model_reject_esfdy = risk_data.model_reject2(df_esfdy, '下单日期')
df_model_reject_esfdy = update_col(df_model_reject_esfdy)
df_model_reject_esfdy.sort_values(by=["下单日期"],inplace=True)
js_esfdy = all_js(df_model_reject_esfdy, df_esfdy_g, list_js)
# js_esfdy

### 32_6.拒量

In [50]:
list_jl = ['命中250513规则1强拒','命中250513规则2强拒','命中250513规则3强拒','命中250513规则4强拒','司法强拒_出库前','命中蚂蚁数控风险等级强拒','命中策略strategy_240927强拒', '命中极信sc32007分强拒', '命中银联模型及Fico联合规则强拒', '命中250513规则5强拒']
df_jl=df_j_risk[(df_j_risk["是否拒量"] == 1)&(df_j_risk["进件"]=="进件")]
df_model_reject_jl = risk_data.model_reject2(df_jl, '下单日期')
df_model_reject_jl = update_col(df_model_reject_jl)
df_model_reject_jl.sort_values(by=["下单日期"],inplace=True)
js_jl = all_js(df_model_reject_jl, df_jl_group, list_jl)
# js_jl

### 32_7.京享租

In [51]:
list_jd = ['司法强拒_机审','司法强拒_出库前','评分等级低于D','命中青云分强拒','命中蚁盾分2.0强拒','命中策略JDB002_212强拒','命中JDB002_202100联合强拒','命中JDB002_202098强拒','命中JDB002_201048强拒','命中京享租信用等级低于D','命中JDB003_rule_1强拒','命中JDB003_rule_2强拒','命中JDB003_rule_4强拒', '命中JDB003_fico强拒']
df_jd=df_j_risk[(df_j_risk["归属渠道"] == "京东渠道")&(df_j_risk["进件"]=="进件")]
df_jd = df_jd[df_jd.下单日期>='2025-05-20']
df_model_reject_jd = risk_data.model_reject2(df_jd, '下单日期')
df_model_reject_jd = update_col(df_model_reject_jd)
df_model_reject_jd.sort_values(by=["下单日期"],inplace=True)
js_jd = all_js(df_model_reject_jd, df_jd_g, list_jd)
# js_jd

## 33.机审出库前仅被该条规则拒绝

In [52]:
# 数据处理
df_j_risk.loc[:, '拒绝理由_new'] = np.where(df_j_risk.拒绝理由.str.contains(',命中拒绝')==True, df_j_risk.拒绝理由.str.replace(',命中拒绝', '命中拒绝'),
                                        np.where(df_j_risk.拒绝理由.str.contains('命中特殊策略2强拒, ')==True, df_j_risk.拒绝理由.str.replace('命中特殊策略2强拒, ', ''), df_j_risk.拒绝理由))
df_j_risk.loc[:, 'rejected_len'] = df_j_risk.拒绝理由_new.apply(lambda x: len(x.split(',')) if x is not None   else x)
df_j_risk_rule = df_j_risk[df_j_risk.rejected_len==1]


### 33_0.机审出库前强拒-all

In [53]:
df_j_risk_rule_all = risk_data.model_reject2(df_j_risk_rule, '下单日期')
df_j_risk_rule_all = update_col(df_j_risk_rule_all)
df_j_risk_rule_all.sort_values(by=["下单日期"],inplace=True)

rule_all = all_js(df_j_risk_rule_all, df_all, list_js2)
# rule_all

### 33_1.芝麻租物

In [54]:
df_zm_rule = df_j_risk_rule[(df_j_risk_rule["归属渠道"]=="芝麻租物")&(df_j_risk_rule["进件"]=="进件")]
df_zm_rule_ = risk_data.model_reject2(df_zm_rule, '下单日期')
df_zm_rule_ = update_col(df_zm_rule_)
df_zm_rule_.sort_values(by=["下单日期"],inplace=True)

rule_zm = all_js(df_zm_rule_, df_zw_g, list_js2)
# rule_zm

### 33_2.搜索渠道

In [55]:
df_ss_rule = df_j_risk_rule[(df_j_risk_rule["归属渠道"]=="搜索渠道")&(df_j_risk_rule["进件"]=="进件")]
df_ss_rule_ = risk_data.model_reject2(df_ss_rule, '下单日期')
df_ss_rule_ = update_col(df_ss_rule_)
df_ss_rule_.sort_values(by=["下单日期"],inplace=True)

rule_ss = all_js(df_ss_rule_, df_ss_g, list_js)
# rule_ss

### 33_3.单人会话

In [56]:
df_dr_rule = df_j_risk_rule[(df_j_risk_rule["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）")&(df_j_risk_rule["进件"]=="进件")]
df_dr_rule_ = risk_data.model_reject2(df_dr_rule, '下单日期')
df_dr_rule_ = update_col(df_dr_rule_)
df_dr_rule_.sort_values(by=["下单日期"],inplace=True)

rule_dr = all_js(df_dr_rule_, df_dr_g, list_js)
# rule_dr

### 33_4.S量

In [57]:
df_sl_rule = df_j_risk_rule[(df_j_risk_rule["归属渠道"]=="S量")&(df_j_risk_rule["进件"]=="进件")]
df_sl_rule_ = risk_data.model_reject2(df_sl_rule, '下单日期')
df_sl_rule_ = update_col(df_sl_rule_)
df_sl_rule_.sort_values(by=["下单日期"],inplace=True)

rule_sl = all_js(df_sl_rule_, df_sl_g, list_s)
# rule_sl

### 33_5.二手非抖音

In [58]:
df_esfdy_rule = df_j_risk_rule[(df_j_risk_rule["归属渠道"]!="抖音渠道")&(df_j_risk_rule.是否二手==1)&(df_j_risk_rule["进件"]=="进件")]
df_esfdy_rule_ = risk_data.model_reject2(df_esfdy_rule, '下单日期')
df_esfdy_rule_ = update_col(df_esfdy_rule_)
df_esfdy_rule_.sort_values(by=["下单日期"],inplace=True)

rule_esfdy = all_js(df_esfdy_rule_, df_esfdy_g, list_js)
# rule_esfdy

### 33_6.拒量

In [59]:
df_jl_rule = df_j_risk_rule[(df_j_risk_rule["是否拒量"] == 1)&(df_j_risk_rule["进件"]=="进件")]
df_j_rule_ = risk_data.model_reject2(df_jl_rule, '下单日期')
df_j_rule_ = update_col(df_j_rule_)
df_j_rule_.sort_values(by=["下单日期"],inplace=True)
rule_jl = all_js(df_j_rule_, df_jl_group, list_jl)
# rule_jl

### 33_7.京享租

In [60]:
df_jd_rule = df_j_risk_rule[(df_j_risk_rule["归属渠道"] == "京东渠道")&(df_j_risk_rule["进件"]=="进件")]
df_jd_rule = df_jd_rule[df_jd_rule.下单日期>='2025-05-20']
df_jd_rule_ = risk_data.model_reject2(df_jd_rule, '下单日期')
df_jd_rule_ = update_col(df_jd_rule_)
df_jd_rule_.sort_values(by=["下单日期"],inplace=True)
rule_jd = all_js(df_jd_rule_, df_jd_g, list_jd)
# rule_jd

# 4.周期数据

In [66]:
week_list = ['week_group', "去重订单数", "前置拦截", "拦截率", "进件数", "预授权通过率", "机审强拒", "机审强拒比例", "机审通过件", "风控通过件","风控通过件比例", "出库前风控强拒", "出库前风控强拒比例", "出库", "进件出库率", "订单出库率"]
all_col_list = ["去重订单数", "前置拦截", "进件数", "机审强拒", "机审通过件", "风控通过件", "出库前风控强拒", "出库"]

def all_week_data(df, key=None):
    df_week = df.set_index('下单日期')[all_col_list]
    week_data = week_models.week_data_group_all(df_week, 7)[week_list]
    if key is not None:
        week_data.insert(1, 'nan1', np.nan)
        week_data.insert(2, 'nan2', np.nan)
    return week_data

# 总体
week_all = all_week_data(df_all, key=1)
# 芝麻租物
week_zw = all_week_data(df_zw_g)
# 搜索渠道
week_ss = all_week_data(df_ss_g)
# 单人会话
week_dr = all_week_data(df_dr_g)
# S量
week_sl = all_week_data(df_sl_g)
# 抖音
week_dy = all_week_data(df_dy_g)
# 二手非抖音
week_esfdy = all_week_data(df_esfdy_g)
# 拒量
df_jl_group2 = df_jl.groupby('下单日期').agg({'order_id':'count', '前置拦截':'sum','是否进件': 'sum', '机审强拒': 'sum', '机审通过件':'sum', '风控通过件':'sum', '出库前风控强拒': 'sum', '是否出库': 'sum'}).rename(columns={'order_id':'去重订单数', '是否进件': '进件数', '是否出库': '出库'})
df_jl_week = df_jl_group2[all_col_list]
week_jl = week_models.week_data_group_all(df_jl_week, 7)[['week_group', '进件数', '机审强拒', '机审强拒比例', '出库前风控强拒', '出库前风控强拒比例', '出库', '进件出库率']]
# 京享租
week_jd = all_week_data(df_jd_g)

# 使用pd.concat拼接
week_alls = pd.concat([week_all, week_zw.iloc[:, 1:], week_ss.iloc[:, 1:], week_dr.iloc[:, 1:], week_sl.iloc[:, 1:], week_dy.iloc[:, 1:], week_esfdy.iloc[:, 1:], week_jl.iloc[:, 1:], week_jd.iloc[:, 1:]], axis=1)
week_alls.columns
    

Index(['week_group', 'nan1', 'nan2', '去重订单数', '前置拦截', '拦截率', '进件数', '预授权通过率',
       '机审强拒', '机审强拒比例',
       ...
       '机审强拒', '机审强拒比例', '机审通过件', '风控通过件', '风控通过件比例', '出库前风控强拒', '出库前风控强拒比例',
       '出库', '进件出库率', '订单出库率'],
      dtype='object', length=130)

In [62]:
# 前置拦截
week_reg_all = week_reg(qz_all, week_all, list_reg[1:], key='前置')
# 芝麻
week_reg_zm = week_reg(qz_zm, week_zw, list_reg[1:], key='前置')
# 搜索
week_reg_ss = week_reg(qz_ss, week_ss, list_reg[1:-2], key='前置')
# 单人会话
week_reg_dr = week_reg(qz_dr, week_dr, list_reg[1:-2], key='前置')
# S量
week_reg_sl = week_reg(qz_sl, week_sl, list_reg[1:-2], key='前置')
# 抖音
week_reg_dy = week_reg(qz_dy, week_dy, list_reg[1:-2], key='前置')
# 二手非抖音
week_reg_esfdy = week_reg(qz_esfdy, week_esfdy, list_reg[1:-2], key='前置')
# 京享租
week_reg_jd = week_reg(qz_jd, week_jd, list_reg[1:-2], key='前置')

In [63]:
# 机审出库前强拒
week_js_all = week_reg(js_all, week_all, list_js2, key='机审')
# 芝麻租物
week_js_zm = week_reg(js_zm, week_zw, list_js2, key='机审')
# 搜索
week_js_ss = week_reg(js_ss, week_ss, list_js, key='机审')
# 单人会话
week_js_dr = week_reg(js_dr, week_dr, list_js, key='机审')
# S量
week_js_sl = week_reg(js_sl, week_sl, list_s, key='机审')
# 二手非抖音
week_js_esfdy = week_reg(js_esfdy, week_esfdy, list_js, key='机审')
# 拒量
week_js_jl = week_reg(js_jl, week_jl, list_jl, key='机审')
# 京享租
week_js_jd = week_reg(js_jd, week_jd, list_jd, key='机审')

In [64]:
# 机审出库前强拒-单条规则
week_rule_all = week_reg(rule_all, week_all, list_js2, key='机审')
# 芝麻租物
week_rule_zm = week_reg(rule_zm, week_zw, list_js2, key='机审')
# 搜索
week_rule_ss = week_reg(rule_ss, week_ss, list_js, key='机审')
# 单人会话
week_rule_dr = week_reg(rule_dr, week_dr, list_js, key='机审')
# S量
week_rule_sl = week_reg(rule_sl, week_sl, list_s, key='机审')
# 二手非抖音
week_rule_esfdy = week_reg(rule_esfdy, week_esfdy, list_js, key='机审')
# 拒量
week_rule_jl = week_reg(rule_jl, week_jl, list_jl, key='机审')
# 京享租
week_rule_jd = week_reg(rule_jd, week_jd, list_jd, key='机审')

In [ ]:
# 免审
df_ms_week = df_ms2_g[["去重订单数", "前置拦截", "进件数", "机审强拒", "机审通过件", '人审拒绝', "风控通过件", '客户取消', '无法联系', "出库前风控强拒", '待审核', "出库"]]
week_ms = week_models.week_data_group_all(df_ms_week, 7)[['week_group', "进件数","人审拒绝","客户取消","出库","待审核","进件出库率","客户取消比例","人审拒绝比例",'出库前风控强拒','出库前风控强拒比例','无法联系','无法联系比例']]
week_ms.insert(1, 'nan1', np.nan)
week_ms.insert(2, 'nan2', np.nan)
# 免押
df_my_week = df_my_g.set_index('下单日期')[["去重订单数", "进件数", "机审强拒", "机审通过件", '人审拒绝', "风控通过件", '客户取消', '待审核', "出库"]]
week_my = week_models.week_data_group_all(df_my_week, 7)[['week_group', "去重订单数","进件数","预授权通过率","机审强拒","机审强拒比例","机审通过件","人审拒绝",
                    "风控通过件","风控通过件比例","客户取消","待审核","出库","进件出库率","客户取消比例","人审拒绝比例","订单出库率"]]
# 非免押
df_fmy_week = df_fmy2_g[["去重订单数", "进件数", "机审强拒", "机审通过件", '人审拒绝', "风控通过件", '客户取消', '待审核', "出库"]]
week_fmy = week_models.week_data_group_all(df_fmy_week, 7)[['week_group', "进件数","机审强拒","机审强拒比例","机审通过件","人审拒绝",
                    "风控通过件","风控通过件比例","客户取消","客户取消比例","人审拒绝比例"]]
# 使用pd.concat进行拼接
week_m_all = pd.concat([week_ms, week_my, week_fmy], axis=1)

# 5.自动写入

In [ ]:
def Open_Excel2(df_list, path,  key2=None):
    '''
    对日报数据进行自动更新
    :param df: 日报各个渠道的df
    :param path: 日报的excel文件路径
    :param password: excel文件的密码
    :param sheet_name: excel的sheet页名称
    :param col: 列的位置，如A列，B列等
    :param start_col: 开始的列数
    :param key2: key2=1则是表示第一次写入，key2=2则是表示写入周期数据
    :return:
    '''
    # 定义文件路径和密码
    file_path = path
    # 使用 xlwings 打开加密的 Excel 文件
    app = xw.App(visible=False)
    # app.display_alerts = False  # 关闭警告提示
    try:
        wb = xw.Book(file_path)
        sheet_names = wb.sheet_names
        start_col=1
        # 获取该工作表的最后一行，并插入一行
        # col_len:判断是否是第一列第一个需要插入的数据；
        # key：值等于1用来判断是否是免人审数据，免审订单转化，押金类型这三个数据页在最后一行的前三行插入； 值不等于1则在最后一行插入并且起始行数最后减4
        # 获取最后一行的日期数据进行判断
        for idx, df_lis in enumerate(df_list):
            # 获取工作表
            sheet = wb.sheets[sheet_names[idx]]
            col = 'C'
            for df in df_lis:
                # 判断是否是写入周数据，写入周数据不需要插入空行
                if key2 == None:
                    last_row = sheet.range(col + str(sheet.cells.last_cell.row)).end('up').row
                    last_date = sheet.range(f'{col}{last_row}').value
                    # 获取前一天的日期
                    now_date = pd.to_datetime((datetime.now()).strftime('%Y-%m-%d'))
                    # 计算当前日期和获取的最大日期的差值
                    diff = (now_date - last_date).days if last_date else 0
                    if now_date != last_date and diff > 0:
                        insert_rows = last_row if diff == 1 else f"{last_row}:{last_row + diff - 1}"
                        sheet.api.Rows(insert_rows).Insert()
                        
                # 重新获取最后一行 在添加周数据的时候用
                last_row2 = sheet.used_range.last_cell.row
                # 获取固定列数的最后一行 在日常数据的时候用
                last_row = sheet.range(col + str(sheet.cells.last_cell.row)).end('up').row
                # 将 DataFrame 写入固定的位置
                # 对应Excel中的行和列，起始行数
                if key2==None:
                    start_row = last_row - len(df) + 1
                elif key2==2:
                    start_row = last_row2 - 2
                else:
                    start_row = last_row + 1
                # 获取上一个df的宽度并相加
                # col_lens = start_col + len(df.columns)
                # 定位写入区域左上角单元格（行, 列）
                target_range = sheet.range((start_row, start_col))
                # 一次性写入整个DataFrame数据
                target_range.options(index=False, header=False).value = df
        # 保存并重新加密文件
        wb.save()
        # return col_lens, start_row
    finally:
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()

In [ ]:
now_day = datetime.now().strftime("%Y%m%d")
# 指定目录路径
directory = 'F:/租后表分析数据_ld/风控周报/风控规则'

# 查找所有 .xlsx 和 .xls 文件
excel_files = glob.glob(os.path.join(directory, '*.xlsx')) + glob.glob(os.path.join(directory, '*.xls'))

for old_file_path in excel_files:
    # 提取文件名（不带路径）
    old_filename = os.path.basename(old_file_path)
    old_date = old_filename.split('.')[0][-8:]
    new_filename = old_filename.replace(old_date, now_day)
    # 将新的文件名添加到路径
    new_file_path = os.path.join(directory, new_filename)

    # 确保新文件名不存在，以避免覆盖文件
    if not os.path.exists(new_file_path):
        os.rename(old_file_path, new_file_path)
        print(f"文件 {old_filename} 已重命名为: {new_filename}")
    else:
        print(f"跳过文件 {old_filename}，因为新文件名已存在。")

文件 1_风控规则命中率日报&月报_20250701 - 副本.xlsx 已重命名为: 1_风控规则命中率日报&月报_2025020250701.xlsx
跳过文件 1_风控规则命中率日报&月报_20250701.xlsx，因为新文件名已存在。


In [ ]:
# path = 'F:/租后表分析数据_ld/风控周报/风控规则命中率日报&月报.xlsx'
# path = f'F:/租后表分析数据_ld/风控周报/风控规则/1_风控规则命中率日报&月报_20250619 -副本.xlsx'
path = f'F:/租后表分析数据_ld/风控周报/风控规则/1_风控规则命中率日报&月报_{now_day}.xlsx' 


In [ ]:
df_list = [[qd_all], [qz_all], [qz_zm], [qz_ss], [qz_dr], [qz_sl], [qz_dy], [qz_esfdy], [qz_jd], [js_all], [js_zm], [rule_all], [rule_zm], [js_ss], [rule_ss], [js_dr], [rule_dr], 
        [js_sl], [rule_sl], [js_esfdy], [rule_esfdy], [js_jl], [rule_jl], [js_jd], [rule_jd], [m_all]] #
Open_Excel2(df_list, path)

df_week_list = [[week_alls], [week_reg_all], [week_reg_zm], [week_reg_ss], [week_reg_dr], [week_reg_sl], [week_reg_dy], [week_reg_esfdy], [week_reg_jd], [week_js_all], [week_js_zm], [week_rule_all], 
                [week_rule_zm], [week_js_ss], [week_rule_ss], [week_js_dr], [week_rule_dr], [week_js_sl], [week_rule_sl], [week_js_esfdy], [week_rule_esfdy], [week_js_jl], [week_rule_jl], [week_js_jd], 
                [week_rule_jd], [week_m_all]]
Open_Excel2(df_week_list, path, key2=2)

In [ ]:
# def run(key2=None):
#     # 1_转化数据
#     # 总体 芝麻 搜索 单人会话 S量 抖音 二手抖音 拒量
#     qd_all = pd.concat([df_all, df_zw_g.iloc[:, 1:], df_ss_g.iloc[:, 1:], df_dr_g.iloc[:, 1:], df_sl_g.iloc[:, 1:], df_dy_g.iloc[:, 1:], df_esfdy_g.iloc[:, 1:], df_jl_group.iloc[:, 1:]], axis=1)
#     qd_all = qd_all.merge(df_jd_g, on='下单日期', how='left')
#     Open_Excel2(qd_all, path, '1_转化数据', 'C', key2=key2) # , key2=1 表示第一次写入

#     # 2_前置拦截
#     Open_Excel2(qz_all, path, '2_前置拦截', 'C', key2=key2)
#     # 21_前置拦截_芝麻租物
#     Open_Excel2(qz_zm, path, '21_前置拦截_芝麻租物', 'C', key2=key2)
#     # 22_前置拦截_搜索渠道
#     Open_Excel2(qz_ss, path, '22_前置拦截_搜索渠道', 'C', key2=key2)
#     # 23_前置拦截_单人会话
#     Open_Excel2(qz_dr, path, '23_前置拦截_单人会话', 'C', key2=key2)
#     # 24_前置拦截_S量
#     Open_Excel2(qz_sl, path, '24_前置拦截_S量', 'C', key2=key2)
#     # 25_前置拦截_抖音
#     Open_Excel2(qz_dy, path, '25_前置拦截_抖音', 'C', key2=key2)
#     # 26_前置拦截_二手非抖音
#     Open_Excel2(qz_esfdy, path, '26_前置拦截_二手非抖音', 'C', key2=key2)
#     # 27_前置拦截_京享租
#     Open_Excel2(qz_jd, path, '27_前置拦截_京享租', 'C', key2=key2)
    
#     # 3_机审出库前规则命中
#     Open_Excel2(js_all, path, '3_机审出库前规则命中', 'C', key2=key2)
#     # 31_机审出库前规则命中_芝麻租物
#     Open_Excel2(js_zm, path, '31_机审出库前规则命中_芝麻租物', 'C', key2=key2)
#     # 32_机审出库前规则命中_搜索渠道
#     Open_Excel2(js_ss, path, '32_机审出库前规则命中_搜索渠道', 'C', key2=key2)
#     # 33_机审出库前规则命中_单人会话
#     Open_Excel2(js_dr, path, '33_机审出库前规则命中_单人会话', 'C', key2=key2)
#     # 34_机审出库前规则命中_S量
#     Open_Excel2(js_sl, path, '34_机审出库前规则命中_S量', 'C', key2=key2)
#     # 35_机审出库前规则命中_二手非抖音
#     Open_Excel2(js_esfdy, path, '35_机审出库前规则命中_二手非抖音', 'C', key2=key2)
#     # 36_机审出库前规则命中_拒量
#     Open_Excel2(js_jl, path, '36_机审出库前规则命中_拒量', 'C', key2=key2)
#     # 37_机审出库前规则命中_京享租
#     Open_Excel2(js_jd, path, '37_机审出库前规则命中_京享租', 'C', key2=key2)
    

#     # 4_机审出库前规则命中_仅单条规则命中
#     Open_Excel2(rule_all, path, '4_机审出库前规则命中_仅单条规则命中', 'C', key2=key2)
#     # 41_机审出库前规则命中_芝麻租物_仅单条规则命中
#     Open_Excel2(rule_zm, path, '41_机审出库前规则命中_芝麻租物_仅单条规则命中', 'C', key2=key2)
#     # 42_机审出库前规则命中_搜索渠道_仅单条规则命中
#     Open_Excel2(rule_ss, path, '42_机审出库前规则命中_搜索渠道_仅单条规则命中', 'C', key2=key2)
#     # 43_机审出库前规则命中_单人会话_仅单条规则命中
#     Open_Excel2(rule_dr, path, '43_机审出库前规则命中_单人会话_仅单条规则命中', 'C', key2=key2)
#     # 44_机审出库前规则命中_S量_仅单条规则命中
#     Open_Excel2(rule_sl, path, '44_机审出库前规则命中_S量_仅单条规则命中', 'C', key2=key2)
#     # 45_机审出库前规则命中_二手非抖音_仅单条规则命中
#     Open_Excel2(rule_esfdy, path, '45_机审出库前规则命中_二手非抖音_仅单条规则命中', 'C', key2=key2)
#     # 46_机审出库前规则命中_拒量_仅单条规则命中
#     Open_Excel2(rule_jl, path, '46_机审出库前规则命中_拒量_仅单条规则命中', 'C', key2=key2)
#     # 47_机审出库前规则命中_京享租_仅单条规则命中
#     Open_Excel2(rule_jd, path, '47_机审出库前规则命中_京享租_仅单条规则命中', 'C', key2=key2)
    

#     # 5_免审&免押
#     # 免审 免押 非免押
#     m_all = pd.concat([df_ms_g, df_my_g, df_fmy_g], axis=1)
#     Open_Excel2(m_all, path, '5_免审&免押', 'C', key2=key2)
# # run(key=1, key2=1)

# run()


In [ ]:
# ## 周数据
# # 1_转化数据
# Open_Excel2(df = week_alls, path = path, sheet_name='1_转化数据', col='C', key2=2)

# # 2_前置拦截
# Open_Excel2(df = week_reg_all, path = path, sheet_name='2_前置拦截', col='C', key2=2)
# Open_Excel2(df = week_reg_zm, path = path, sheet_name='21_前置拦截_芝麻租物', col='C', key2=2)
# Open_Excel2(df = week_reg_ss, path = path, sheet_name='22_前置拦截_搜索渠道', col='C', key2=2)
# Open_Excel2(df = week_reg_dr, path = path, sheet_name='23_前置拦截_单人会话', col='C', key2=2)
# Open_Excel2(df = week_reg_sl, path = path, sheet_name='24_前置拦截_S量', col='C', key2=2)
# Open_Excel2(df = week_reg_dy, path = path, sheet_name='25_前置拦截_抖音', col='C', key2=2)
# Open_Excel2(df = week_reg_esfdy, path = path, sheet_name='26_前置拦截_二手非抖音', col='C', key2=2)
# Open_Excel2(df = week_reg_jd, path = path, sheet_name='27_前置拦截_京享租', col='C', key2=2)

# # 3_机审出库前规则命中
# Open_Excel2(df = week_js_all, path = path, sheet_name='3_机审出库前规则命中', col='C', key2=2)
# Open_Excel2(df = week_js_zm, path = path, sheet_name='31_机审出库前规则命中_芝麻租物', col='C', key2=2)
# Open_Excel2(df = week_js_ss, path = path, sheet_name='32_机审出库前规则命中_搜索渠道', col='C', key2=2)
# Open_Excel2(df = week_js_dr, path = path, sheet_name='33_机审出库前规则命中_单人会话', col='C', key2=2)
# Open_Excel2(df = week_js_sl, path = path, sheet_name='34_机审出库前规则命中_S量', col='C', key2=2)
# Open_Excel2(df = week_js_esfdy, path = path, sheet_name='35_机审出库前规则命中_二手非抖音', col='C', key2=2)
# Open_Excel2(df = week_js_jl, path = path, sheet_name='36_机审出库前规则命中_拒量', col='C', key2=2)
# Open_Excel2(df = week_js_jd, path = path, sheet_name='37_机审出库前规则命中_京享租', col='C', key2=2)

# # 4_机审出库前规则命中_仅单条规则命中
# Open_Excel2(df = week_rule_all, path = path, sheet_name='4_机审出库前规则命中_仅单条规则命中', col='C', key2=2)
# Open_Excel2(df = week_rule_zm, path = path, sheet_name='41_机审出库前规则命中_芝麻租物_仅单条规则命中', col='C', key2=2)
# Open_Excel2(df = week_rule_ss, path = path, sheet_name='42_机审出库前规则命中_搜索渠道_仅单条规则命中', col='C', key2=2)
# Open_Excel2(df = week_rule_dr, path = path, sheet_name='43_机审出库前规则命中_单人会话_仅单条规则命中', col='C', key2=2)
# Open_Excel2(df = week_rule_sl, path = path, sheet_name='44_机审出库前规则命中_S量_仅单条规则命中', col='C', key2=2)
# Open_Excel2(df = week_rule_esfdy, path = path, sheet_name='45_机审出库前规则命中_二手非抖音_仅单条规则命中', col='C', key2=2)
# Open_Excel2(df = week_rule_jl, path = path, sheet_name='46_机审出库前规则命中_拒量_仅单条规则命中', col='C', key2=2)
# Open_Excel2(df = week_rule_jd, path = path, sheet_name='47_机审出库前规则命中_京享租_仅单条规则命中', col='C', key2=2)

# # 5_免审&免押
# Open_Excel2(df = week_m_all, path = path, sheet_name='5_免审&免押', col='C', key2=2)